In [12]:
import glob

import iris
from iris.experimental.equalise_cubes import equalise_attributes

import warnings
warnings.filterwarnings('ignore')

import xarray as xr

from dask.distributed import Client, progress, LocalCluster

In [2]:
client = LocalCluster() 
client

In [16]:
model = 'NorESM1-M'
thetao_files = '/g/data/ua6/DRSv2/CMIP5/%s/piControl/mon/ocean/r1i1p1/thetao/latest/thetao_Omon_%s_piControl_r1i1p1_*.nc' %(model, model)
thetao_file_list = glob.glob(thetao_files)

## xarray

In [17]:
ds = xr.open_mfdataset(thetao_files)
thetao = ds['thetao']

In [18]:
thetao

<xarray.DataArray 'thetao' (time: 6012, lev: 70, j: 384, i: 320)>
dask.array<shape=(6012, 70, 384, 320), dtype=float32, chunksize=(48, 70, 384, 320)>
Coordinates:
  * lev      (lev) float64 0.0 5.0 10.0 15.0 20.0 25.0 30.0 40.0 50.0 62.5 ...
  * j        (j) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
  * i        (i) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
    lat      (j, i) float32 -79.22052 -79.22052 -79.22052 -79.22052 ...
    lon      (j, i) float32 320.5625 321.6875 322.8125 323.9375 325.0625 ...
  * time     (time) object 0700-01-16 12:00:00 0700-02-15 00:00:00 ...
Attributes:
    standard_name:     sea_water_potential_temperature
    long_name:         Sea Water Potential Temperature
    units:             K
    original_name:     templvl
    original_units:    degC
    cell_methods:      time: mean
    cell_measures:     area: areacello volume: volcello
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [19]:
# the number of GB involved in the reduction
ds.thetao.nbytes/1e9

206.8512768

In [20]:
thetao.mean().load()

<xarray.DataArray 'thetao' ()>
array(281.2781, dtype=float32)

## iris

In [25]:
cube = iris.load(thetao_files, 'sea_water_potential_temperature')
equalise_attributes(cube)
cube = cube.concatenate_cube()
cube

<iris 'Cube' of sea_water_potential_temperature / (K) (time: 6012; depth: 70; cell index along second dimension: 384; cell index along first dimension: 320)>

In [26]:
coord_names = [coord.name() for coord in cube.dim_coords]
coord_names

['time',
 'depth',
 'cell index along second dimension',
 'cell index along first dimension']

In [28]:
cube.collapsed(coord_names, iris.analysis.MEAN).data

array(281.27866, dtype=float32)